## Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Наша задача - найти взаимосвязь между количеством детей, семейным положением и добросовестным исполнением обязанностей заемщика.

Из информации о загруженной базе данных от банка мы видим, что у нас есть гораздо больше столбцов с данными, чем нам нужно, к тому же, в них есть пропуски. мы могли бы бороть все их, но это пустая трата времени, т.к. в нашем исследовании мы попрасту не будем обращаться к ним, поэтому, на мой взлягд, после стадии определения дубликатов нам просто стоит удалить из БД столбцы, которые нам не понадобятся.

P.S. Если мы удалим их ДО, то могут возникнуть "лжедубликаты" которые будут индентичны в оставшихся столбцах, но в удаленных они могли бы отличаться.

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Обработка пропусков в столбце "количество детей в семье"

In [2]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце с информацией о кол-ве детей есть явная проблема с двумя из восьми уникальных значений. Если количество детей от 0 до 5 нас вполне устраивает, то на значениях "-1" и "20" стоит заострить внимание. С подобными значениями мы работать явно не можем, да и корректную информацию о количестве детей данных зайемщиков нам взять неоткуда.

Общее число подобных случаев составляет 76 + 47 = 123, т.е. 123 / 21525 = 0.57% от общих данных. Думаю, таким количеством данных мы можем пожертвовать и просто удалить её из нашей БД:

In [3]:
data.drop(data[data.children == -1].index, inplace = True)
data.drop(data[data.children == 20].index, inplace = True)
data = data.reset_index(drop = True)

Мы удалили строки с заведомо ошибочными значениями в столбце информации о детях, проверим, какие уникальные значения в этом столбце у нас остались:

In [4]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

#### Обработка пропусков в столбцах "семейное положение" и "идентификатор семейного положения"

Теперь проверим на пропуски вторую группу важных для нас столбцов - "Семейное положение" и "идентификатор семейного положения":

In [5]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [6]:
data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

В столбе с информацией о семейном положении найдено 5 вариантов ответа и все выглядят подходящими и логичными. Сделаем вывод, что в данном столбе необходимость в работе с пропусками отсутствует.
Кол-во текстовых вариантов совпадает с кол-вом ID семейных положений.

#### Обработка пропусков в столбце "ежемесячный доход"

Большое количество пропусков в информации о доходах заемщиков оставить без внимания нельзя, ведь нам необходимо оценить платежеспособность клиентов и  их возможность погашать кредит в срок, а как это сделать, если не по их доходам. Так как получить достоверную информацию у нас не получится, попробуем присвоить пропускам средние/медиальные значения для схожих типов занятости.

Для начала посмотрим средние и медиальные значения ежемесячного дохода для каждого типа занятости:

In [7]:
avg_salary = data.groupby('income_type')['total_income'].mean()
avg_salary

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170743.143230
компаньон          202585.823720
пенсионер          137198.468121
предприниматель    499163.144947
сотрудник          161365.012384
студент             98201.625314
Name: total_income, dtype: float64

In [8]:
median_salary = data.groupby('income_type')['total_income'].median()
median_salary

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150420.150276
компаньон          172517.418907
пенсионер          118480.837408
предприниматель    499163.144947
сотрудник          142587.588976
студент             98201.625314
Name: total_income, dtype: float64

Для всех типов занятости (с большим количеством представителей) медиальные значения всегда ниже чем средние. Можно сделать вывод, что в предложенной статистике присутствуют "богачи" искажающие значения среднего дохода, поэтому я буду заполнять пропуски медиальными значениями:

In [9]:
data['total_income'] = data['total_income'].fillna(value = 0.0) 
#data.loc[(data['total_income'] == 0.0) & (data['income_type'] == 'госслужащий'), 'total_income'] = median_salary[2]
#data.loc[(data['total_income'] == 0.0) & (data['income_type'] == 'компаньон'), 'total_income'] = median_salary[3]
#data.loc[(data['total_income'] == 0.0) & (data['income_type'] == 'пенсионер'), 'total_income'] = median_salary[4]
#data.loc[(data['total_income'] == 0.0) & (data['income_type'] == 'предприниматель'), 'total_income'] = median_salary[5]
#data.loc[(data['total_income'] == 0.0) & (data['income_type'] == 'сотрудник'), 'total_income'] = median_salary[6]
#data.info()

In [10]:
n = -1
for row in data['total_income']:
    n += 1
    if row == 0.0:
        for i in median_salary.index:
            if data.loc[n, 'income_type'] == i:
                data.loc[n, 'total_income'] = median_salary[i]

Теперь все заемщики имеют доход, хоть и не все - реальный.

**Вывод**

Не всегда стоит уделять время пропускам, если они никак не повлияют на результат нашей работы. Однако, если есть пропуски в важной для нашей работы информации, стоит задать себе несколько вопросов: 
1. Можем ли мы где-то взять утерянную/испорченную иформацию?
2. Если пропуски присутствуют в количественных переменных, можем ли мы взять среднее или медиальное значение?
3. Можем ли мы пожертвовать "испорценными" данными?

Чем позже мы ответим "да", тем менее точной получится наша работа, но, в любом случае, это будет лучше, чем оперировать заведомо ошибочной информацией.

### Замена типа данных

In [11]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       19240 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null float64
purpose             21402 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Один из наших столбоцов - "ежемесячный доход" необоснованно имеет формат вещественного числа, считая заработную плату вплоть до десятитысячных копейки. Пользы от этого мало, но лишняя информация загромождает таблицы и память нашего компьютера, давайте избавимся от ненужной мелочи:

In [12]:
data['total_income'] = data['total_income'].astype('int')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       19240 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Может кто-то и любит большие числа в графе "заработная плата", но наша валюта не так крепка, чтобы дорожить долями копеек.

### Обработка дубликатов

Любая система может дать сбой и записать одного и того же клиента несколько раз, поэтому стоит проверить нашу базу данных на подобные ошибки.

Если посмотреть на первые пять строк нашей таблицы, мы уже можем обратить внимание, что в столбце "уровень образования клиента" регистр букв отличается. Стоит привести его к единому:

In [13]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [14]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

С этим разобрались, столбец "цель получения кредита" тоже не вызывает доверия и есть причины полагать, что после лемматизации у нас обнаружатся новые дубликаты, но это чуть позже, пока попробуем сбросить полностью дублирующиеся строки:

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       19240 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [16]:
data = data.drop_duplicates()
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 12 columns):
children            21331 non-null int64
days_employed       19240 non-null float64
dob_years           21331 non-null int64
education           21331 non-null object
education_id        21331 non-null int64
family_status       21331 non-null object
family_status_id    21331 non-null int64
gender              21331 non-null object
income_type         21331 non-null object
debt                21331 non-null int64
total_income        21331 non-null int64
purpose             21331 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Как мы видим, в наших данных нашлись 71 полный дубликат строки!
Теперь, предположим, что заемщик, при заполнении анкеты, случайно ошибся в одном из пунктов, но заметил ошибку только после отправки данных и решил заполнить форму еще раз. Поищим дубликаты без учета каждого из столбцов по одному (за исключением таких как "общий трудовой стаж в днях" и "ежемесячный доход" т.к. эти показатели крайне индививдуальны и имеют огромный диапазон возможных значений, и, если их убрать, могут найтись дествительно одинаковые клиенты).

Начнем с пола:

In [17]:
data = data.drop_duplicates(subset = ['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'income_type', 'debt', 'total_income', 'purpose'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21299 entries, 0 to 21298
Data columns (total 12 columns):
children            21299 non-null int64
days_employed       19240 non-null float64
dob_years           21299 non-null int64
education           21299 non-null object
education_id        21299 non-null int64
family_status       21299 non-null object
family_status_id    21299 non-null int64
gender              21299 non-null object
income_type         21299 non-null object
debt                21299 non-null int64
total_income        21299 non-null int64
purpose             21299 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Еще 32 строки отсеилось. На мой взгляд, крайне маловероятно, что в банке могли встретиться на столько похожие люди с единственным отличием!

Теперь проверим, есть ли люди, не сразу вспомнили сколько у них детей:

In [18]:
data = data.drop_duplicates(subset = ['days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21264 entries, 0 to 21263
Data columns (total 12 columns):
children            21264 non-null int64
days_employed       19240 non-null float64
dob_years           21264 non-null int64
education           21264 non-null object
education_id        21264 non-null int64
family_status       21264 non-null object
family_status_id    21264 non-null int64
gender              21264 non-null object
income_type         21264 non-null object
debt                21264 non-null int64
total_income        21264 non-null int64
purpose             21264 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


Еще 35 человек с одинаковым стажем до дня и заработной платой до рубля, не говоря уже о возрасте, семейном положении и и цели кредита, но отличающихся лишь по количеству детей.

Проверим по возрасту:

In [19]:
data = data.drop_duplicates(subset = ['children', 'days_employed', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20616 entries, 0 to 20615
Data columns (total 12 columns):
children            20616 non-null int64
days_employed       19240 non-null float64
dob_years           20616 non-null int64
education           20616 non-null object
education_id        20616 non-null int64
family_status       20616 non-null object
family_status_id    20616 non-null int64
gender              20616 non-null object
income_type         20616 non-null object
debt                20616 non-null int64
total_income        20616 non-null int64
purpose             20616 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


Уоу, а это уже пугает. Неужели 562 человека могли "накосячить" с возрастом при заполнении анкеты?

А что с ошибками в графе "образование"?

In [20]:
data = data.drop_duplicates(subset = ['children', 'days_employed', 'dob_years', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20608 entries, 0 to 20607
Data columns (total 12 columns):
children            20608 non-null int64
days_employed       19240 non-null float64
dob_years           20608 non-null int64
education           20608 non-null object
education_id        20608 non-null int64
family_status       20608 non-null object
family_status_id    20608 non-null int64
gender              20608 non-null object
income_type         20608 non-null object
debt                20608 non-null int64
total_income        20608 non-null int64
purpose             20608 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


Всего 8 человек, не так плохо.

А сколько человек с первого раза не справилось с семейным положением?

In [21]:
data = data.drop_duplicates(subset = ['children', 'days_employed', 'dob_years', 'education', 'education_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20583 entries, 0 to 20582
Data columns (total 12 columns):
children            20583 non-null int64
days_employed       19240 non-null float64
dob_years           20583 non-null int64
education           20583 non-null object
education_id        20583 non-null int64
family_status       20583 non-null object
family_status_id    20583 non-null int64
gender              20583 non-null object
income_type         20583 non-null object
debt                20583 non-null int64
total_income        20583 non-null int64
purpose             20583 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


И еще 25.

А с типом занятости?

In [22]:
data = data.drop_duplicates(subset = ['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'debt', 'total_income', 'purpose'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20583 entries, 0 to 20582
Data columns (total 12 columns):
children            20583 non-null int64
days_employed       19240 non-null float64
dob_years           20583 non-null int64
education           20583 non-null object
education_id        20583 non-null int64
family_status       20583 non-null object
family_status_id    20583 non-null int64
gender              20583 non-null object
income_type         20583 non-null object
debt                20583 non-null int64
total_income        20583 non-null int64
purpose             20583 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


Никого, это радует.

А вот теперь мы дошли до очень важного момента: я уверен, что мы можем найти дубликаты, если проанализируем базу данных без учета столбца "имел ли задолженность по возврату кредитов", но этот параметр нам крайне важен для анализа, а какой из дубликатов правдивый, а какой нет, мы никак не сможем узнать, поэтому если мы, например, удалим те, в которых значится НАЛИЧИЕ задолженостей, а оставим с их остутсвием, что, по факту, не будет являться действительностью, мы своими руками повлияем на точность нашего анализа.

Остается последний вариант, проверка без учета столбца "цель получения кредита". Даже если один и тот же человек подкает 2 заявки с разными целями кредита, для нас он все равно дубликат, т.к. мы оцениваем именно людей, а не заявки.

In [23]:
data = data.drop_duplicates(subset = ['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income'])
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20354 entries, 0 to 20353
Data columns (total 12 columns):
children            20354 non-null int64
days_employed       19240 non-null float64
dob_years           20354 non-null int64
education           20354 non-null object
education_id        20354 non-null int64
family_status       20354 non-null object
family_status_id    20354 non-null int64
gender              20354 non-null object
income_type         20354 non-null object
debt                20354 non-null int64
total_income        20354 non-null int64
purpose             20354 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


Нашлось до 229 человек, которые подали заявку на кредит более одного раза!
P.S. "До" - потому что может быть и более 2 заявков от одного лица.

**Вывод**

К поиску дубликатов стоит подходить основательно, т.к. информация, которую мы анализируем, заполняется простыми людми, которые склонны ошибаться. Если отличие присутствует лишь в одном параментре, но все остальные, включая заработную плату или трудовой стаж схожи вплоть до копейки или трудового дня, стоит задуматься - возможно ли такое?

P.S. прошу заметить, что отключение изменения формата столбца "total_income" из float64 в int64 путем "закомментирования" кода и перезапуска всей страницы не влияет на полученные результаты.

### Лемматизация

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Для анализа нам может потребоваться объединение заявок на кредит в группы и отличным критерием может выступить "цель получения кредита", но, как мы видим выше, цели описаны в свободной форме и, при разной формулировке, по сути уникальных не так уж много, поэтому нам стоит сгруппировать их путем лемматизации.

После анализа уникальных значений столбца "цель получения кредита" можно выделить 6 основных целей:
1. Жилье
2. Автомобиль 
3. Образование
4. Свадьба
5. Недвижимость
6. Ремонт

P.S. на мой взгляд, категоризация по "цели получения кредита" для нашего исследования бесполезна, поэтому цель моего выполнения этого задания: практика и демонстрация усвоения навыка лемматизации.

In [25]:
from pymystem3 import Mystem
m = Mystem()
def lemms(text):
    lemm_text = m.lemmatize(text)
    if 'ремонт' in lemm_text:
        return 'ремонт'
    if 'жилье' in lemm_text:
        return 'жилье'
    if 'недвижимость' in lemm_text:
        return 'недвижимость'
    if 'автомобиль' in lemm_text:
        return 'автомобиль'
    if 'образование' in lemm_text:
        return 'образование'
    if 'свадьба' in lemm_text:
        return 'свадьба'

Когда речь идет о свадьбе - деньги необходимы на организацию, об образовании - на его оплату, об автомобиле - на его покупку, о недвижимости - на покупку или, как выражаются заявители, на операции с ней, но вот среди целей, связанных с жильем, нашлась одна, которая заключалась не в преобретении, а в ремонте, поэтому я решил выделить данную цель в отдельную категорию. Так как слово ремонт всегда сопровождается словом жилье, то я решил эту проблему разместив проверку на слово "ремонт" на первое место. Таким образом заявки, касающиеся ремонта жилья помечяются словом "ремонт", а остальные уходят в "жилье".

In [26]:
data['purpose'] = data['purpose'].apply(lemms)
data.info()
data['purpose'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20354 entries, 0 to 20353
Data columns (total 12 columns):
children            20354 non-null int64
days_employed       19240 non-null float64
dob_years           20354 non-null int64
education           20354 non-null object
education_id        20354 non-null int64
family_status       20354 non-null object
family_status_id    20354 non-null int64
gender              20354 non-null object
income_type         20354 non-null object
debt                20354 non-null int64
total_income        20354 non-null int64
purpose             20354 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 1.9+ MB


недвижимость    6023
автомобиль      4100
образование     3812
жилье           3663
свадьба         2179
ремонт           577
Name: purpose, dtype: int64

**Вывод**

Лемматизация помогает сгруппировать одинаковые значения, описанные разными словами, что, в свою очередь, упрощает процесс категоризации для дальнейшего анализа базы данных.

### Категоризация данных

Категоризация данных поможет нам отсечь все лишнее и оставить лишь интересующую в каждом отдельном вопросе информацию.

Для оценки влияния количества детей на наличие/отсутствие задолженностей по кредитам отделим эти столбцы и сгруппируем по количеству детей:

In [27]:
children_debt = data[['children', 'debt',]]
children_debt = children_debt.groupby('children').mean().sort_values('debt',ascending = False)
children_debt['debt'] = children_debt['debt'] * 100
children_debt['debt'] = children_debt['debt'].astype('str') + '%'
children_debt

,debt
children,
4,10.0%
2,9.722921914357682%
1,9.540254694582345%
3,8.256880733944955%
0,7.8293413173652695%
5,0.0%


В результате мы получили таблицу, где в первом столбце информация о кол-ве детей у заемщика, а справа процент людей с соответствующим кол-вом детей и задолженностью по кредиту.

Теперь сгруппируем по семейному статусу

In [28]:
family_status_debt = data[['family_status', 'debt',]]
family_status_debt = family_status_debt.groupby('family_status').mean().sort_values('debt',ascending = False)
family_status_debt['debt'] = family_status_debt['debt'] * 100
family_status_debt['debt'] = family_status_debt['debt'].astype('str') + '%'
family_status_debt

,debt
family_status,
Не женат / не замужем,10.08869179600887%
гражданский брак,9.534648457258472%
женат / замужем,7.877744296168747%
в разводе,7.210300429184549%
вдовец / вдова,6.892778993435448%


Слева - семейное положение, справа - процент людей с данным семейным положением и с задолженностью по кредиту.

Теперь изменим параметр, по которому мы группируем. Давайте узнаем, есть ли разница в месячном окладе людей, у которых когда-то была задолженнность по кредиту и тех, кто всегда платил в срок:

In [29]:
total_income_debt = data[['total_income', 'debt',]]
total_income_debt.groupby('debt').mean().sort_values('total_income',ascending = False)

,total_income
debt,
0,166869.629385
1,161423.683995


И, наконец, сделаем таблицу, показывающую какой процент людей имел задолженность по кредиту в зависимости от целей этого кредита:

In [30]:
purpose_debt = data[['purpose', 'debt',]]
purpose_debt = purpose_debt.groupby('purpose').mean().sort_values('debt',ascending = False)
purpose_debt['debt'] = purpose_debt['debt'] * 100
purpose_debt['debt'] = purpose_debt['debt'].astype('str') + '%'
purpose_debt

,debt
purpose,
автомобиль,9.634146341463415%
образование,9.548793284365162%
свадьба,8.077099586966497%
недвижимость,7.803420222480491%
жилье,7.425607425607425%
ремонт,6.065857885615252%


#### <font color='purple'> Категоризация по количеству детей

In [31]:
def howmanychildren (children):
    if children == 0:
        return 'бездетный'
    if children > 2:
        return 'многодетный'
    return 'есть дети'    

In [32]:
data['how_many_children'] = data['children'].apply(howmanychildren)
children_debt_2 = data[['how_many_children', 'debt',]]
children_debt_2 = children_debt_2.groupby('how_many_children').mean().sort_values('debt',ascending = False)
children_debt_2['debt'] = children_debt_2['debt'] * 100
children_debt_2['debt'] = children_debt_2['debt'].astype('str') + '%'
children_debt_2

,debt
how_many_children,
есть дети,9.595043819885161%
многодетный,8.24468085106383%
бездетный,7.8293413173652695%


In [33]:
data['total_income'].min()

20667

In [34]:
data['total_income'].max()

2265604

In [35]:
data['total_income'].mean()

166411.56421342242

In [36]:
data['total_income'].median()

143966.0

In [37]:
data.sort_values(by = 'total_income', ascending = False).head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,how_many_children
11885,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2265604,ремонт,бездетный
18578,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2200852,недвижимость,есть дети
8830,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,образование,есть дети
19686,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,жилье,бездетный
16331,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1711309,свадьба,бездетный
16630,0,-2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1597613,недвижимость,бездетный
17443,1,-333.935516,41,высшее,0,гражданский брак,1,M,компаньон,0,1551152,свадьба,есть дети
17430,1,-3173.282035,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1427934,автомобиль,есть дети
14563,1,-10207.448165,64,высшее,0,в разводе,3,M,компаньон,0,1350245,жилье,есть дети
10624,1,-1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1286280,недвижимость,есть дети


In [38]:
data.sort_values(by = 'total_income', ascending = False).tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,how_many_children
17716,1,-1953.777603,50,среднее,1,женат / замужем,0,F,госслужащий,0,29200,автомобиль,есть дети
3957,0,386389.497407,64,среднее,1,женат / замужем,0,F,пенсионер,1,29154,недвижимость,бездетный
19132,0,381266.917841,59,среднее,1,женат / замужем,0,F,пенсионер,0,29067,недвижимость,бездетный
15796,0,-241.247456,49,среднее,1,гражданский брак,1,F,компаньон,0,28702,свадьба,бездетный
14244,2,-1440.353367,28,среднее,1,гражданский брак,1,F,сотрудник,0,28092,свадьба,есть дети
20043,0,360310.974923,55,среднее,1,Не женат / не замужем,4,M,пенсионер,0,27907,жилье,бездетный
10207,0,385664.024835,67,среднее,1,женат / замужем,0,F,пенсионер,0,27776,образование,бездетный
16986,0,338590.212074,59,среднее,1,женат / замужем,0,F,пенсионер,0,27414,образование,бездетный
7989,0,-615.774147,46,среднее,1,женат / замужем,0,F,сотрудник,0,26533,недвижимость,бездетный
14676,1,-2945.051315,45,среднее,1,женат / замужем,0,F,сотрудник,0,26329,недвижимость,есть дети


In [39]:
def worth (salary):
    if salary < 100000:
        return 'бедные'
    if 100000 <= salary < 145000:
        return 'класс ниже среднего'
    if 145000 <= salary < 250000:
        return 'класс выше среднего'
    return 'богатые'

In [40]:
data['worth'] = data['total_income'].apply(worth)
#data.groupby('worth').count()
total_income_debt_2 = data[['debt', 'worth']]
total_income_debt_2 = total_income_debt_2.groupby('worth').mean().sort_values('debt',ascending = False)
total_income_debt_2['debt'] = total_income_debt_2['debt'] * 100
total_income_debt_2['debt'] = total_income_debt_2['debt'].astype('str') + '%'
total_income_debt_2

,debt
worth,
класс ниже среднего,9.554683501109027%
класс выше среднего,8.328736900165472%
бедные,7.965796579657966%
богатые,6.936002860207364%


**Вывод**

Обилие различных данных не всегда помогает проанализировать ситуацию. Гораздо удобнее выделить из таблицы только те данные, которые нам необходимы для ответа на поставленный вопрос.

## Шаг 3. Ответы на вопросы

***- Есть ли зависимость между наличием детей и возвратом кредита в срок?***

In [41]:
children_debt

,debt
children,
4,10.0%
2,9.722921914357682%
1,9.540254694582345%
3,8.256880733944955%
0,7.8293413173652695%
5,0.0%


Как мы видим, 7.8% клиентов имеют задолженности по кредитам, но у клиентов с 1-4 ребенком этот увеличивается до 8.3-10% и лишь заемщики с 5 детьми вообще не имеют задолженности по кредитам, в наше время такое большое количество детей - большая редкость, поэтому предположу, что подборка крайне мала и данный показатель не стоит считать точным. Давайте убедимся, что таких людей единицы:

In [42]:
data['children'].value_counts()

0    13360
1     4633
2     1985
3      327
4       40
5        9
Name: children, dtype: int64

Всего 9 человек с 5 детьми, не стоит доверять статистике, основанной на 9 случаях.

Наличие детей негативно сказывается на возможности погашать кредит вовремя.

***- Есть ли зависимость между семейным положением и возвратом кредита в срок?***

In [43]:
family_status_debt

,debt
family_status,
Не женат / не замужем,10.08869179600887%
гражданский брак,9.534648457258472%
женат / замужем,7.877744296168747%
в разводе,7.210300429184549%
вдовец / вдова,6.892778993435448%


Из таблицы выше явно видно, что холостые клиенты - самые ненадежные, аж 10.1% имели дело с просрочкой по кредиту, в то время как люди, живущие в отношениях более склонны возвращать долг вовремя, причем чем официальнее отношения, тем ниже процент проскрочки: гражданские браки 9.5%, а официальный уже 7.9%. И самыми лучшими клиентами оказались люди, имевшие серьезные отношения, но по каким либо причинам их потерявшие: Клиенты в разводе 7.2%, а вдовцы/вдовы всего 6.9%.

**Вывод**

Чем более зрелый человек в вопросах образования семьи, тем, как оказалось, он к кредитам подходит ответственней. 

Для более подробного анализа мы можем использовать сводную таблицу ниже и определить вероятность просрочки по кредиту для людей, например, в гражданском браке и с 3 детьми - целых 14.6%!!!

In [44]:
data.pivot_table(index=['family_status'], columns='children', values='debt', aggfunc='mean')

children,0,1,2,3,4,5
family_status,,,,,,
Не женат / не замужем,0.096552,0.116331,0.121622,0.125000,0.500000,NaN
в разводе,0.071802,0.068627,0.086420,0.090909,0.000000,NaN
вдовец / вдова,0.065432,0.090909,0.150000,0.000000,0.000000,NaN
гражданский брак,0.085901,0.121622,0.087613,0.145455,0.000000,0.0
женат / замужем,0.072009,0.086237,0.098039,0.068826,0.107143,0.0


***- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?***

In [45]:
total_income_debt.groupby('debt').mean().sort_values('total_income',ascending = False)

,total_income
debt,
0,166869.629385
1,161423.683995


Если рассматривать статистику о средней зарплате клиентов с задолженностями и без, то у вторых хоть она и выше, но всего на 3.4%. Это и не удивительно, но давайте посмотрим статистику, разделив клиентов еще по типу занятости:

In [46]:
data.pivot_table(index=['income_type'], columns='debt', values='total_income', aggfunc='mean')

debt,0,1
income_type,,
безработный,202722.000000,59956.000000
в декрете,NaN,53829.000000
госслужащий,170154.552651,152014.348837
компаньон,201240.074916,193932.319035
пенсионер,136325.451183,136803.725118
предприниматель,499163.000000,NaN
сотрудник,160939.757791,155738.449038
студент,98201.000000,NaN


<font color='purple'> **Новый ответ начинается здесь.**

In [47]:
total_income_debt_2

,debt
worth,
класс ниже среднего,9.554683501109027%
класс выше среднего,8.328736900165472%
бедные,7.965796579657966%
богатые,6.936002860207364%


<font color='purple'> В результате исследования мы узнали, что самыми надежными заемщиками, как ни странно, являются богатые люди (с достатоком выше 250 тыс. рублей в месяц), а вот наименее надежные - средний класс, причем присутствует значительная разница между средним классом с з.п. ниже общей медиальной (60 тыс. руб. - 145 тыс. руб.) и выше медиальной. Более бедный средний класс на 1.22% чаще имеет задолженоость по кредиту. Что же касается бедных, то они, на удивление, оказались более ответственными, чем средний класс, с вероятностью 7.97% просрочить оплату по кредиту.

**Вывод**

Даже при более подробном анализе мы видим, что заработная плата всех, у кого были/есть долгт по кредитам ниже относительно коллег без долгов. Однако, благодаря этой таблицы мы можем сделать вывод, что возможность вернуть кредит в срок у пенсионеров не зависит от месячного дохода.

<font color='purple'> Вывод выше прошу считать недействительным)
    
**Новый вывод**
    
Существует чёткая зависимость между доходом клиента и шансом, что он нарушит свои обязательства, однако, изменение этого шанса не имеет прямого отношения к изменению месячного дохода, как могло казаться изначально.

***- Как разные цели кредита влияют на его возврат в срок?***

In [48]:
purpose_debt

,debt
purpose,
автомобиль,9.634146341463415%
образование,9.548793284365162%
свадьба,8.077099586966497%
недвижимость,7.803420222480491%
жилье,7.425607425607425%
ремонт,6.065857885615252%


Самый большой риск - предоставлять кредиты для покупки автомобиля (9.6% клиентов с просрочкой) или оплаты образования (9.5% клиентов с просрочкой).

Клиенты, берущие кредит на организацию свадьбы в 8.1% случаев не оплачивают платежи в срок.

Те, кто берут крупные суммы на покупку жилья и недвижимости немного более надежные клиенты, чем перечисленные выше, их шанс по неисполнению обязанностей 7.4% и 7.8% соответственно.

И самыми надежными клиентами оказались ремонтирующие своё жилье (6.1% просрочки)

**Вывод**

Какой либо закономерности здесь нет, можем лишь воспользоваться знаниями горького опыта прошлых клиентов.

## Шаг 4. Общий вывод

Мы выявили четкую закономерность между наличием детей или их отсутствием и частотой нарушений обязанностей по кредиту, а вот от количества детей этот показатель логичной закономерности не показал. С семейным положением все более прозрачно: чем более серьезно человек относится к своему партнеру и отношениям, тем добросовестнее он относится и к финансам. Доходы же имеют не такое сильное влияние на ответственность заемщиков, но, в любом случае, чем выше заработная плата, тем выше шанс, что клиент будет добросовестно исполнять обязанности.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.